**Cylindrical Thermal Convection**

Compared to *Effects of Curvature on Two-Dimensional Models of Mantle Convection: Cylindrical Polar Coordinates* (Jarvis, 1993)

Jarvis found relationships for Nu and average temperature with the cylindrical mesh geometry, specifically the ratio of outer to inner radii, f.

**Limitations:** The models of Jarvis have free outer and inner surfaces, whereas these are fixed in this model. Here, free surfaces are *approximated* by including low viscosity inner and outer layers. This results in a better agreement with Jarvis' models.


In [ ]:
# Whether or not to start from the previous temperature field
loaddata = True

In [ ]:
import underworld as uw
import glucifer
import numpy as np
from underworld import function as fn
uw.matplotlib_inline()
import math

In [ ]:
import matplotlib.pyplot as plt
plt.ion()

In [ ]:
annulus = uw.mesh._FeMesh_Annulus(elementRes=(32,64), 
                                  radialLengths=(0.4292,1.4292), angularExtent=(0.,360.),
                                  periodic = [False, True])


tField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)
vField = uw.mesh.MeshVariable(annulus, nodeDofCount=2)
pField = uw.mesh.MeshVariable(annulus.subMesh, nodeDofCount=1)

In [ ]:
Ra = 1e5

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Mesh(annulus, segmentsPerEdge=1))
fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True ))
fig.show()
# fig.show()

In [ ]:
jWalls = annulus.specialSets["MinJ_VertexSet"] + annulus.specialSets["MaxJ_VertexSet"]

In [ ]:
t_outer = 0.0
t_inner = 1.0



outer = annulus.specialSets["MaxI_VertexSet"]
inner = annulus.specialSets["MinI_VertexSet"]




In [ ]:

# setup parameters for temperature distribution
dr = annulus.radialLengths[1] - annulus.radialLengths[0]
dT_dr = (t_outer-t_inner)/(dr)
c0 = t_inner - dT_dr*annulus.radialLengths[0]

# wavenumber for perturbation
k = 3.5

for ind,coord in enumerate(annulus.data):
    r = np.sqrt(coord[0]**2 + coord[1]**2)
    theta = np.arctan2(coord[1], coord[0])
    
    pert = 0.2 *np.sin(k*theta)
    
    t = r*dT_dr + c0
    tField.data[ind] = min([max([0.,t + 1.*pert]),1])
    
tField.data[inner.data] = t_inner
tField.data[outer.data] = t_outer
    
if loaddata:
    tField.load('Jarvis93_freesurf_data/temp.h5',interpolate=True)

In [ ]:
# A hacky  way to import data from a different resolution, as the loaddata interpolation
# doesn't like the inner and outer surfaces of the cylindrical geometry


importlowres = False

if importlowres:
    tempannulus = uw.mesh._FeMesh_Annulus(elementRes=(16,32), 
                                      radialLengths=(0.4292,1.4292), angularExtent=(0.,360.),
                                      periodic = [False, True])
    tempTfield = uw.mesh.MeshVariable(tempannulus, nodeDofCount=1)
    tempTfield.load('Jarvis93_freesurf_data/temp.h5')

    for node in range(annulus.nodesGlobal):
        pos = annulus.data[node]
        if node not in inner and node not in outer:
            tField.data[node] = tempTfield.evaluate(tuple(pos))


In [ ]:
fig.show()

In [ ]:
vBC = uw.conditions.DirichletCondition( variable=vField, indexSetsPerDof=(inner+outer, inner+outer))
tBC = uw.conditions.DirichletCondition( variable=tField, indexSetsPerDof=(inner+outer))

In [ ]:
tDotField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)
advDiffSLE = uw.systems.AdvectionDiffusion(tField, tDotField, vField, fn_diffusivity=1.0, conditions=tBC)

In [ ]:
g  = 1.0*annulus.fn_uvec_radial()

In [ ]:
bodyForceFn = g * tField * Ra / (annulus.radialLengths[1]-annulus.radialLengths[0])

In [ ]:
# Low viscosity inner and outer layers
yCoord = uw.function.input()
rCoord = uw.function.math.dot(yCoord,yCoord)
bL = 0.1
wVisc = 1e-2
viscMap = uw.function.branching.conditional([(rCoord <= (bL + annulus.radialLengths[0])**2., wVisc),
                                            (rCoord >= (-bL + annulus.radialLengths[1])**2., wVisc),
                                            (True, 1.)])

In [ ]:
stokesSLE = uw.systems.Stokes( vField, pField, fn_viscosity=viscMap, fn_bodyforce=bodyForceFn, conditions=vBC)

In [ ]:
stokesSolver = uw.systems.Solver(stokesSLE)

In [ ]:
nS = 10
arrT = np.zeros(nS)
for i in range(nS):
    stokesSolver.solve()


    dt = 0.5 * advDiffSLE.get_max_dt()
    advDiffSLE.integrate(dt)
    
    arrT[i] = np.average(tField.data[:])

plt.plot(np.arange(nS),arrT)
plt.xlabel("Time Steps")
plt.ylabel("Average Temperature")
plt.ylim([0.25,0.5])
plt.savefig('avtemp.pdf')


**Compare quantities from Jarvis 1993**

In [ ]:
f = annulus.radialLengths[0] / annulus.radialLengths[1]
TavPred = 1. / (1. + f**-0.75)

print("For f=%.2f, Jarvis 93 predict t_av = %.3f" %(f,TavPred) )
print("Modelled t_av = %.3f" %np.average(tField.data[:]))

In [ ]:

D = 2. * np.pi
# Number of up-wellings
n = 3
# Aspect parameter, defined by Jarvis 93
A =  (np.pi/2./n)*(1.+f)/(1.-f)
print("Apect parameter used here is A=%.2f, compared to Jarvis A=1" %A)
print("For A=1 and f = 0.3, Jarvis found Nu=8.4  (This was approximately read from Fig. 7)")



In [ ]:
tgrad = fn.math.dot(tField.fn_gradient,annulus.fn_uvec_radial())
tSurf_integral  = uw.utils.Integral( mesh=annulus, fn=tgrad, integrationType="surface", surfaceIndexSet=outer )
Nu = -1 * tSurf_integral.evaluate()[0] / (2.*np.pi *annulus.radialLengths[1])
print("From our model, Nu = %.2f" %Nu)

The average temperatures is very close to that of Jarvis 1993. Our Nu is much close to theirs, compared to if the pseudo-free surface weak layers are absent. The agreement is likely to be better at higher resolutions.

In [ ]:
fig.show()

In [ ]:
figV = glucifer.Figure(**fig)
figV.append(glucifer.objects.Surface(annulus, tField,onMesh=True))

figV.append(glucifer.objects.VectorArrows(annulus, vField / Ra  * 10. , onMesh=True,arrowHead = 0.3))

figV.show()
figV.save_image('temperature.png')

In [ ]:
# Save data for restart

mH = annulus.save("Jarvis93_freesurf_data/mesh.h5")
tF = tField.save('Jarvis93_freesurf_data/temp.h5',mH)